# IDRNN
[Description]
(Add description here)

The goal of the project:
1. palceholder
2. placeholder


# Dataset

First download the dataset.

# Packages

In [ ]:
# !pip3 install -r requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import  Dataset, DataLoader
import torch.nn.functional as F



# load data and print it

In [ ]:
# from dataloader import ProteinDataset, proteinread
from dataloader import AminoAcidDataset

In [ ]:
# Replace this path with the actual path to your input CSV file
input_csv_file = "data.csv"

# # Create an instance of the ProteinDataset
# train_data, seq_length = proteinread(input_csv_file)

# # Create dataset instances for training and validation
# train_data = ProteinDataset(train_data)

# # Create data loaders for training and validation
# batch_size = 1  # You can adjust this batch size
# train_dataset = DataLoader(train_data, batch_size=batch_size, shuffle=True)

max_sequence_length = 1000
num_amino_acids = 20

dataset = AminoAcidDataset(input_csv_file, max_sequence_length, num_amino_acids)

# Create a DataLoader
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Iterate through the DataLoader
# for batch in dataloader:
#     print("Batch shape:", batch.shape)
#     print("Batch data:", batch)
#     break



In [ ]:
# pick up one sequence to show how our data looks like
g, g_len, target = dataset[65]
print(g)
print(g_len)
print(target)

In [ ]:

# Create an empty list to store the numerical sequences
training_sequences = []
# Iterate over the training_dataset
for sample in dataset:
    # Extract the numerical part (the first element of the tuple) and append it to the list
    training_sequences.append(sample[0])

# Print the list of training sequences
print(training_sequences[65])

In [ ]:
print(training_sequences)

# Helper function and calculation (kld reconstruct loss) and printer form tuorial 6

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

In [ ]:
## Help function from tutroial 6 to calculate loss and plotting

# def sample(mean, log_var):
#     std = torch.exp(0.5*log_var)
#     eps = torch.randn_like(std, device=mean.device)
#     return mean + eps*std
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params
def plot_losses(reco_losses, kld_losses):
    fig = plt.figure(figsize=(10, 5))

    ax = fig.add_subplot(1, 2, 1)
    ax.plot(reco_losses)
    ax.set_title('Reconstruction Loss')

    ax = fig.add_subplot(1, 2, 2)
    ax.plot(kld_losses)
    ax.set_title('KLD Loss')
# def generate_seq(decoder_nn, num_seq=6):
#     new_z = torch.randn(num_seq, 3)
#     if torch.cuda.is_available():
#         new_z = new_z.cuda()

#     new_pred = decoder_nn(new_z)

#     new_pred = new_pred.cpu().detach().numpy()

#     for i in range(32):
#         ax = fig.add_subplot(gs[i])
#         img_array = new_pred[i][0]
#         ax.imshow(img_array, cmap='gray')
#         ax.axis('off')
#         if cond:
#             ax.set_title(f'Cond input: {rand_int[i]}')
def plot_z_dist(train_dl, encoder_nn, cond=False):
    fig = plt.figure(figsize=(24, 6))
    gs = GridSpec(1, 4)


    mus = []; log_vars = []; zs = []; ys = []
    for xb, yb, yb_one_hot in train_dl:
        if torch.cuda.is_available():
            xb = xb.cuda()
            yb_one_hot = yb_one_hot.cuda()

        if cond:
            mu, log_var = encoder_nn(xb, yb_one_hot)
        else:
            mu, log_var = encoder_nn(xb)
        z = sample(mu, log_var)

        mu = mu.cpu().detach().numpy()
        log_var = log_var.cpu().detach().numpy()
        z = z.cpu().detach().numpy()

        mus.extend(mu.tolist())
        log_vars.extend(log_var.tolist())
        zs.extend(z.tolist())
        ys.extend(yb.tolist())

    zs = np.array(zs)
    mus = np.array(mus)
    log_vars = np.array(log_vars)
    ys = np.array(ys)


    ax = fig.add_subplot(gs[0])
    ax.scatter(zs[:, 0], zs[:, 1], c=ys, cmap='tab10', alpha=0.3, s=1)
    ax.set_title('Z')
    ax.grid()
    ax.set_aspect('equal')

    gauss0 = np.random.normal(0, 1, len(zs[:, 0]))
    gauss1 = np.random.normal(0, 1, len(zs[:, 0]))
    ax = fig.add_subplot(gs[1])
    ax.scatter(gauss0, gauss1, c='cornflowerblue', alpha=0.3, s=1)
    ax.set_title('Gaus(0,1))')
    ax.set_aspect('equal')

    return ys, zs

# trainning loop based on tutorial 6

In [ ]:
from model_VAE import VAEGPT, Vaeln

latent_dim = 32

# vae = VAEGPT(input_size=21, output_size=21, latent_dim=latent_dim)

hid_layer = [1000, 500, 100, 32]
vae = Vaeln(input_dim=21*1000, latent_dim=latent_dim, hid_layer=hid_layer)

# 看我派出cuda把你们都送上天！
if torch.cuda.is_available():
    vae.cuda()

In [ ]:
print(f'VAE has {count_parameters(vae):,} trainable parameters')

In [ ]:

optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

beta = 1 # Weight of the KL divergence term


def vae_loss(recon_x, x, mu, logvar):
    # Reconstruction loss using cross-entropy

    cross_entropy_loss = F.cross_entropy(recon_x.permute(0, 2, 1), x.argmax(dim=2), reduction='mean')

    # Regularization term (KL divergence)
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    # Combine reconstruction loss and regularization term
    total_loss = cross_entropy_loss + (beta * kl_divergence)
    
    return kl_divergence, total_loss

num_epochs = 10

for epoch in tqdm(range(num_epochs)):
    vae.train()
    total_losses = []
    # reco_losses = []
    kld_losses  = []
    for seq_batch, length_batch, target_batch in dataloader:
        if torch.cuda.is_available():
            seq_batch = seq_batch.cuda()
            length_batch = length_batch.cuda()
            target_batch = target_batch.cuda()
        optimizer.zero_grad()
        
        recon_batch, mean, logvar = vae(seq_batch)
        kld_loss, total_loss = vae_loss(recon_batch, seq_batch, mean, logvar)
        
        total_loss.backward()
        # torch.nn.utils.clip_grad_norm_(vae.parameters(), max_norm=10000)
        optimizer.step()
        total_losses.append(total_loss.cpu().detach().numpy())
        # reco_losses.append(reco_loss.cpu().detach().numpy())
        kld_losses.append(kld_loss.cpu().detach().numpy())
        # total_loss += loss.item()
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")


In [ ]:

plot_losses(reco_losses, kld_losses)

Test an example

In [ ]:

# Function to convert one-hot encoding to amino acid sequences
def one_hot_to_sequence(one_hot):
    _, max_indices = torch.max(one_hot, dim=1)
    amino_acids = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y", ""]
    sequences = []
    for indices in max_indices:
        sequence = "".join([amino_acids[idx] if idx < len(amino_acids) else '-' for idx in indices if idx != 0])
        sequences.append(sequence)
    return sequences

# Test the trained model on an example from the training dataset
test_batch_size = 1  # Set batch size for testing
test_dataloader = DataLoader(dataset, batch_size=test_batch_size, shuffle=True)

# Retrieve an example from the test dataset
example_seq, example_len, example_target = next(iter(test_dataloader))

# Move the input tensor to the same device as the model
device = next(vae.parameters()).device
example_seq = example_seq.to(device)
example_len = example_len.to(device)

# Set the model to evaluation mode
vae.eval()

# Perform inference on the example
with torch.no_grad():
    recon_output, _, _ = vae(example_seq)
    recon_output = torch.softmax(recon_output.permute(0,2,1), dim=1)

# Convert the output to amino acid sequences
amino_acid_sequences = one_hot_to_sequence(recon_output)

# Print the reconstructed sequences
for i, sequence in enumerate(amino_acid_sequences):
    print(f"Example {i+1}: {sequence}")

# Save the trained model as a .pt file
model_state_dict = vae.state_dict()  # Get the model's state_dict
device = next(vae.parameters()).device  # Get the device of the model

# Move the model's state_dict to the CPU if it's currently on a GPU
if device.type == 'cuda':
    model_state_dict = {k: v.to('cpu') for k, v in model_state_dict.items()}

# Save the model's state_dict to a .pt file
torch.save(model_state_dict, "trained_vae_model.pt")

In [ ]:
original_input = one_hot_to_sequence(example_seq.transpose(1,2))
for i, sequence in enumerate(original_input):
    print(f"Original {i+1}: {sequence}")
print(example_seq)
print(recon_output)